```sql
movielens_development=# select  ARRAY[1,2,4] && ARRAY[3];
 ?column?
----------
 f
(1 行记录)

movielens_development=# select  ARRAY[1,2,4] && ARRAY[2,3,4];
 ?column?
----------
 t
(1 行记录)
```


## Cross join

    T1 CROSS JOIN T2

    For every possible combination of rows from T1 and T2 (i.e., a Cartesian product), the joined table will contain a row consisting of all columns in T1 followed by all columns in T2. If the tables have N and M rows respectively, the joined table will have N * M rows.

    FROM T1 CROSS JOIN T2 is equivalent to FROM T1 INNER JOIN T2 ON TRUE (see below). It is also equivalent to FROM T1, T2.

```sql
select * from pinned_tweets where user_handle = 'rey';
--  tweet_id | user_handle | pinned_at
-- ----------+-------------+-----------
-- (0 rows)

insert into pinned_tweets (user_handle, tweet_id, pinned_at)
  values (
    'rey',
    5,
    clock_timestamp()
  )
on conflict (user_handle)
do update set (tweet_id, pinned_at) = (5, clock_timestamp())
where pinned_tweets.user_handle = 'rey';
-- INSERT 0 1

select * from pinned_tweets where user_handle = 'rey';
--  tweet_id | user_handle |          pinned_at
-- ----------+-------------+------------------------------
--         5 | rey         | 2016-01-11 17:08:52.41554-06
-- (1 row)

```

The addition of the upsert feature means we don't have to tangle with writeable CTEs here. Instead, we get to write a standard insert statement. All we have to add is the on conflict clause. The on conflict clause is a way to tell the insert statement what to do if it is colliding with an existing record. In our case, if the user_handle conflicts with an existing record in the pinned_tweets table, then we react by doing an update instead. That's an upsert.



## conflict Action



    conflict_action specifies an alternative ON CONFLICT action. It can be either DO NOTHING, or a DO UPDATE clause specifying the exact details of the UPDATE action to be performed in case of a conflict. The SET and WHERE clauses in ON CONFLICT DO UPDATE have access to the existing row using the table's name (or an alias), and to rows proposed for insertion using the special excluded table. SELECT privilege is required on any column in the target table where corresponding excluded columns are read.

    Note that the effects of all per-row BEFORE INSERT triggers are reflected in excluded values, since those effects may have contributed to the row being excluded from insertion.


https://ruby-china.org/topics/37522